# ${\textbf{G1: Generating a dataframe of all $H_s$ SWOT - SARWAVE colocated points}}$  

We generate a dataframe (and save it to save time) with the following characteristics:

df_hs = pd.DataFrame({"hs_swot_on_iw": [], # SWOT interpolated hs on sarwave's grid  
                      "hsml_iw": [], # $H_s$ most likely   
                      "conf_iw": [], # confidence index (SARWAVE)   
                      "swot_file": [], # swot file path  
                      "iw_file": [], # iw file path  
                      "lon":[], longitude of SARWAVE's current grid cell  
                      "lat":[], latitude of SARWAVE's current grid cell    
                      "lon_map_min": [], # Corners of the current co-located case (englobes the whole iw-swot overlapping area)
                      "lon_map_max": [],  
                      "lat_map_min": [],  
                      "lat_map_max": []})  


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import SWOT_tools as st
import xarray as xr
import shapefile as shp 
import os
from datetime import datetime
from tqdm import tqdm
from scipy.interpolate import griddata
from scipy.ndimage import binary_dilation
from time import time

import pandas as pd
#plt.rc('font',**{'family':'sans-serif','sans-serif':['Times New Roman']})

In [5]:
import matplotlib.font_manager as fm
# Regular and bold font paths
font_regular_path = "fonts/times/times.ttf"
font_bold_path = "fonts/times/timesbd.ttf"  # Bold
# Register both fonts
fm.fontManager.addfont(font_regular_path)
fm.fontManager.addfont(font_bold_path)
# Create font properties
font_regular = fm.FontProperties(fname=font_regular_path)
font_bold = fm.FontProperties(fname=font_bold_path)
# Set default font to regular Times New Roman
plt.rcParams['font.family'] = font_regular.get_name()

In [6]:
df_coloc = pd.read_csv("G_df/df_colocs_swot_S1.csv")
df_coloc

,Unnamed: 0,year,day_of_year,swot_file,iw_file
0,0,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
1,1,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
2,2,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
3,0,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
4,1,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
...,...,...,...,...,...
10813,1,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
10814,2,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
10815,0,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN
10816,1,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,NaN


In [7]:
# On vire les rows où le iw_file n'est pas encore processé en L2 (nan value)
df_coloc = df_coloc.dropna(subset=["iw_file"])
df_coloc

,Unnamed: 0,year,day_of_year,swot_file,iw_file
9,0,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
10,1,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
11,2,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
18,0,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
19,1,2023.0,263.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
...,...,...,...,...,...
10762,1,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
10763,2,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
10764,0,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...
10765,1,2025.0,1.0,/home/datawork-WW3/PROJECT/SWOT/WindWave/SWOT_...,/home/datawork-cersat-public/cache/project/sar...


In [8]:
swot_unique_file_path = np.unique(df_coloc["swot_file"])

MEAN_HSML = []
MEAN_HSML_CONFIDENT = []
MEAN_HS_SWOT = []

df_hs = pd.DataFrame({"hs_swot_on_iw": [],
                      "hsml_iw": [], # "hsml_confident_iw": [],
                      "conf_iw": [],
                      "swot_file": [],
                      "iw_file": [],
                      "lon":[],
                      "lat":[],
                      "lon_map_min": [],
                      "lon_map_max": [],
                      "lat_map_min": [],
                      "lat_map_max": []})

N = len(swot_unique_file_path)
t0 = time()


for i, swot_file_path in enumerate(swot_unique_file_path):
    df_swot_file_i = df_coloc.loc[df_coloc["swot_file"] == swot_file_path]

    lon_map_min, lon_map_max = 180, -180 # valeurs extrêmes exprès pour que ça se mette bien
    lat_map_min, lat_map_max = 90, -90


    HSML = [] 
    HSML_CONFIDENT = []
    for iw_file in df_swot_file_i["iw_file"]:
        ds = xr.open_dataset(iw_file, group='intraburst')
        
        lon, lat = ds.variables["longitude"].values, ds.variables["latitude"].values
        
        if np.nanmin(lon) < lon_map_min:
            lon_map_min = np.nanmin(lon)
        if np.nanmax(lon) > lon_map_max:
            lon_map_max = np.nanmax(lon)
        if np.nanmin(lat) < lat_map_min:
            lat_map_min = np.nanmin(lat)
        if np.nanmax(lat) > lat_map_max:
            lat_map_max = np.nanmax(lat)
        
        hsml = ds.variables["hs_most_likely"].values
        HSML.append(np.nanmean(hsml))

        hs_conf = ds.variables["hs_conf"].values
        conf_rate_min = -1
        mask_confident = hs_conf > conf_rate_min # hs_conf [-1,1] so 0 is 50%-50%, -1 is 100 % (but then we can filter later)
        hsml_confident = np.ones(hsml.shape)*np.nan
        hsml_confident[mask_confident] = hsml[mask_confident]
        HSML_CONFIDENT.append(np.nanmean(hsml_confident))

    MEAN_HSML.append(np.nanmean(HSML))
    MEAN_HSML_CONFIDENT.append(np.nanmean(HSML_CONFIDENT))
    
    ####################
    ### LOADING SWOT ###
    ####################
    
    ### Launching ###
    ds_swot = xr.open_dataset(swot_file_path)
    try:
        ds_swot, t_ini_swot_formatted, t_end_swot_formatted = st.format_ds_swot(ds_swot, lon_map_min, lon_map_max, lat_map_min, lat_map_max, untrustable_hs=40, kernel_size_nan=1, step_to_crop_at_edges=2)
    except AttributeError: # ça arrive quand le fichier swot est corrompu et qu'il y a des nan partout
        MEAN_HS_SWOT.append(np.nan)
        continue # on va au swot file d'après
    except IndexError:
        continue

    #print(f"lon_map_min = {lon_map_min}; lon_map_max = {lon_map_max}; lat_map_min = {lat_map_min}, lat_map_max = {lat_map_max}")
    lon_swot = ds_swot.longitude.values
    lat_swot = ds_swot.latitude.values
    hs_swot = ds_swot.swh_karin.values

    ### Maintenant on interpole
    points = np.concatenate((lon_swot.flatten().reshape(-1,1), lat_swot.flatten().reshape(-1,1)),axis=1) # shape (n, 2): coordonnées de SWOT 

    for iw_file in df_swot_file_i["iw_file"]:
        ds = xr.open_dataset(iw_file, group='intraburst')
        # On récupère la grille lon, lat de l'iw
        x_grid, y_grid = ds.variables["longitude"].values, ds.variables["latitude"].values

        #interpolated_values_on_grid = griddata(points, values, (grid_x, grid_y), method='linear') # ligne de code pour se rappeler comment ça marche
        hs_swot_on_iw = griddata(points, hs_swot.flatten(), (x_grid, y_grid), method='linear')

        hsml = ds.variables["hs_most_likely"].values

        hs_conf = ds.variables["hs_conf"].values
        conf_rate_min = -1
        mask_confident = hs_conf > conf_rate_min # hs_conf [-1,1] so 0 is 50%-50%
        hsml_confident = np.ones(hsml.shape)*np.nan
        hsml_confident[mask_confident] = hsml[mask_confident]

        idx_no_nan = np.where(~np.isnan(hs_swot_on_iw) & ~np.isnan(hsml_confident))  # e.g. tuple (66, 2) (2 pour x, y)

        
        df_hs_i = pd.DataFrame({"hs_swot_on_iw": hs_swot_on_iw[idx_no_nan].flatten(), # e.g. shape (66,)
                                "hsml_iw": hsml[idx_no_nan].flatten(), # e.g. shape (66,)  "hsml_confident_iw": hsml_confident[idx_no_nan].flatten(), # e.g. shape (66,)
                                "conf_iw": hs_conf[idx_no_nan].flatten(),
                                "swot_file": [swot_file_path for k in range(idx_no_nan[0].shape[0])], # e.g. len (66,)
                                "iw_file": [iw_file for k in range(idx_no_nan[0].shape[0])],
                                "lon": x_grid[idx_no_nan].flatten(),
                                "lat": y_grid[idx_no_nan].flatten(),
                                "lon_map_min": [lon_map_min for k in range(idx_no_nan[0].shape[0])],
                                "lon_map_max": [lon_map_max for k in range(idx_no_nan[0].shape[0])],
                                "lat_map_min": [lat_map_min for k in range(idx_no_nan[0].shape[0])],
                                "lat_map_max": [lat_map_max for k in range(idx_no_nan[0].shape[0])]})
                               
                               
        if not df_hs_i.empty:
            df_hs = pd.concat((df_hs, df_hs_i))
    
    MEAN_HS_SWOT.append(np.nanmean(hs_swot))
    print(f"Progress: {round(100*i/N,2)} % ----- Elapsed time: {(time() - t0)//60} m {round((time() - t0)%60,0)} s")
    

df_hs.to_csv("G_df/df_hs_with_iw_files.csv")


/dev/shm/pbs.3818094.datarmor0/ipykernel_51728/2776775329.py:114: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_hs = pd.concat((df_hs, df_hs_i))


Progress: 0.33 % ----- Elapsed time: 0.0 m 3.0 s
Progress: 0.66 % ----- Elapsed time: 0.0 m 5.0 s
Progress: 0.98 % ----- Elapsed time: 0.0 m 7.0 s
Progress: 1.31 % ----- Elapsed time: 0.0 m 8.0 s
Progress: 1.64 % ----- Elapsed time: 0.0 m 11.0 s
Progress: 1.97 % ----- Elapsed time: 0.0 m 13.0 s
Progress: 2.3 % ----- Elapsed time: 0.0 m 15.0 s
Progress: 2.62 % ----- Elapsed time: 0.0 m 18.0 s
Progress: 2.95 % ----- Elapsed time: 0.0 m 20.0 s
Progress: 3.28 % ----- Elapsed time: 0.0 m 23.0 s
Progress: 3.61 % ----- Elapsed time: 0.0 m 25.0 s
Progress: 3.93 % ----- Elapsed time: 0.0 m 26.0 s
Progress: 4.26 % ----- Elapsed time: 0.0 m 28.0 s
Progress: 4.92 % ----- Elapsed time: 0.0 m 35.0 s
Progress: 5.25 % ----- Elapsed time: 0.0 m 38.0 s
Progress: 5.57 % ----- Elapsed time: 0.0 m 46.0 s
Progress: 5.9 % ----- Elapsed time: 0.0 m 48.0 s
Progress: 6.23 % ----- Elapsed time: 0.0 m 50.0 s
Progress: 6.56 % ----- Elapsed time: 0.0 m 52.0 s
Progress: 7.21 % ----- Elapsed time: 0.0 m 58.0 s
Progre

/dev/shm/pbs.3818094.datarmor0/ipykernel_51728/2776775329.py:116: RuntimeWarning: Mean of empty slice
  MEAN_HS_SWOT.append(np.nanmean(hs_swot))


Progress: 9.18 % ----- Elapsed time: 1.0 m 8.0 s
Progress: 9.51 % ----- Elapsed time: 1.0 m 10.0 s
Progress: 9.84 % ----- Elapsed time: 1.0 m 11.0 s
Progress: 10.16 % ----- Elapsed time: 1.0 m 13.0 s
Progress: 10.49 % ----- Elapsed time: 1.0 m 15.0 s
Progress: 10.82 % ----- Elapsed time: 1.0 m 17.0 s


/dev/shm/pbs.3818094.datarmor0/ipykernel_51728/2776775329.py:116: RuntimeWarning: Mean of empty slice
  MEAN_HS_SWOT.append(np.nanmean(hs_swot))


...

Progress: 92.46 % ----- Elapsed time: 34.0 m 48.0 s
Progress: 92.79 % ----- Elapsed time: 34.0 m 52.0 s
Progress: 93.11 % ----- Elapsed time: 34.0 m 56.0 s
Progress: 93.77 % ----- Elapsed time: 35.0 m 1.0 s
Progress: 94.1 % ----- Elapsed time: 35.0 m 5.0 s
Progress: 94.43 % ----- Elapsed time: 35.0 m 8.0 s


/dev/shm/pbs.3818094.datarmor0/ipykernel_51728/2776775329.py:116: RuntimeWarning: Mean of empty slice
  MEAN_HS_SWOT.append(np.nanmean(hs_swot))


Progress: 95.08 % ----- Elapsed time: 35.0 m 12.0 s
Progress: 95.41 % ----- Elapsed time: 35.0 m 15.0 s


/home1/datahome/msimonne/scripts/Codes_PFE_Maturin/SWOT_tools.py:245: RuntimeWarning: All-NaN slice encountered
  t_ini_swot = np.nanmin(ds_swot.time.values)


Progress: 96.07 % ----- Elapsed time: 35.0 m 22.0 s
Progress: 96.39 % ----- Elapsed time: 35.0 m 25.0 s
Progress: 97.38 % ----- Elapsed time: 35.0 m 34.0 s
Progress: 97.7 % ----- Elapsed time: 35.0 m 36.0 s
Progress: 98.03 % ----- Elapsed time: 35.0 m 40.0 s
Progress: 98.36 % ----- Elapsed time: 35.0 m 44.0 s
Progress: 98.69 % ----- Elapsed time: 35.0 m 48.0 s
Progress: 99.02 % ----- Elapsed time: 35.0 m 53.0 s
Progress: 99.67 % ----- Elapsed time: 35.0 m 57.0 s
